In [24]:
!pip install azure-common==1.1.28
!pip install azure-core==1.26.2
!pip install azure-digitaltwins-core==1.2.0
!pip install azure-identity==1.12.0
!pip install azure-mgmt-authorization==3.0.0
!pip install azure-mgmt-core==1.3.2
!pip install azure-mgmt-digitaltwins==6.4.0
!pip install azure-mgmt-resource==22.0.0
!pip install pandas

  Using cached azure_mgmt_authorization-3.0.0-py3-none-any.whl.metadata (14 kB)
Using cached azure_mgmt_authorization-3.0.0-py3-none-any.whl (965 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\MAHE\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\azure\\mgmt\\authorization\\v2018_05_01_preview\\aio\\operations\\_access_review_instances_assigned_for_my_approval_operations.py'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



Upload models


In [25]:

# Credentials
DThostname = "https://ADT-FYP-smarthomes-name.api.wcus.digitaltwins.azure.net"

In [26]:
import json
import os

directory = "./models/"
list_of_models = []

for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)
     
        with open(file_path, 'r') as f:
            model = json.load(f)
            list_of_models.append(model)


In [27]:
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential

# Replace the instance url of your digital twin here
adtInstanceUrl = f"{DThostname}"

dt_client = DigitalTwinsClient(adtInstanceUrl, DefaultAzureCredential())
print("Service Client Created! Ready to go")
list_of_models
try:
    existing_models = dt_client.list_models()
    for model in existing_models:
        dt_client.delete_model(model.id)
    print("Existing models deleted")
except Exception as e:
    print("Error deleting existing models:", e)
try:
    # put all the models that we want to create into a list
    models = list_of_models
    dt_client.create_models(models)
    print("Model uploaded to the instance")
except Exception as e:
    print(e)

Service Client Created! Ready to go
Existing models deleted
Model uploaded to the instance


In [28]:
# DT Create

import pandas as pd

def delete_all_relationships():
    query_expression = 'SELECT * FROM digitaltwins'
    query_result = dt_client.query_twins(query_expression)  # get all twins
    print('DigitalTwins:')
    for twin in query_result:
        # print(twin)
        digital_twin_id = twin['$dtId']
        relationships = dt_client.list_relationships(digital_twin_id) # list the twins relationships
        for relationship in relationships:
            relationship_id = relationship['$relationshipId']
            dt_client.delete_relationship(digital_twin_id, relationship_id) # delete the relationship
            print(f'Deleted: {relationship} from {digital_twin_id}')

def delete_all_digital_twins():
    query_expression = 'SELECT * FROM digitaltwins'
    query_result = dt_client.query_twins(query_expression)
    for twin in query_result:
        digital_twin_id = twin['$dtId']
        dt_client.delete_digital_twin(digital_twin_id)
        print(f'Deleted twin: {digital_twin_id}')
        
def create_digital_twins_from_excel(file_path):

    delete_all_relationships()
    delete_all_digital_twins()
    # Read the Excel file
    df = pd.read_excel(file_path)
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Extract model ID, digital twin ID, and metadata from the row
        model_id = row['ModelID']
        digital_twin_id = row['ID (must be unique)']
        metadata = {
            "$metadata": {
                "$model": model_id
            },
            "$dtId": digital_twin_id
        }
        
        # Upsert the digital twin
        created_twin = dt_client.upsert_digital_twin(digital_twin_id, metadata)
        
        # Print a message indicating success
        print(f"Digital twin with ID {digital_twin_id} created for model {model_id}")

# Example usage:
file_path = 'Room-sensordata.xlsx'
create_digital_twins_from_excel(file_path)



DigitalTwins:
Deleted twin: Room1
Deleted twin: AC1
Deleted twin: TV1
Deleted twin: Fan1
Deleted twin: Geyser1
Deleted twin: Fridge1
Deleted twin: Bulb1
Deleted twin: Washer1
Deleted twin: Thermostat1
Digital twin with ID Room1 created for model dtmi:example:Room;1
Digital twin with ID AC1 created for model dtmi:example:AC;1
Digital twin with ID TV1 created for model dtmi:example:TV;1
Digital twin with ID Fan1 created for model dtmi:example:Fan;1
Digital twin with ID Geyser1 created for model dtmi:example:Geyser;1
Digital twin with ID Fridge1 created for model dtmi:example:Fridge;1
Digital twin with ID Bulb1 created for model dtmi:example:LightBulb;1
Digital twin with ID Washer1 created for model dtmi:example:WashingMachine;1
Digital twin with ID Thermostat1 created for model dtmi:example:Thermostat;1


In [34]:
# Delete all twins and relationships
delete_all_relationships()
delete_all_digital_twins()

DigitalTwins:
Deleted twin: AC1
Deleted twin: TV1
Deleted twin: Fan1
Deleted twin: Geyser1
Deleted twin: Fridge1
Deleted twin: Bulb1
Deleted twin: Washer1
Deleted twin: Thermostat1
Deleted twin: Room1


In [35]:
# Delete all models
try:
    existing_models = dt_client.list_models()
    for model in existing_models:
        dt_client.delete_model(model.id)
    print("Existing models deleted")
except Exception as e:
    print("Error deleting existing models:", e)

Existing models deleted


In [29]:
import pandas as pd
import random
import time
from datetime import datetime
import os
import json

# Function to update the parameters with random values
def update_parameters(df):
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Update Init Data parameters
        init_data = json.loads(row['Init Data'])
        for key in init_data.keys():
            # Update only the values of Init Data parameters
            init_data[key] = get_random_value(key)
        df.at[index, 'Init Data'] = json.dumps(init_data)
    return df

# Function to get a random value based on the parameter key
def get_random_value(key):
    if key == 'temperature':
        return random.randint(18, 28)
    elif key == 'mode':
        return random.choice(["Cool", "Normal", "Heat"])
    elif key == 'fanSpeed':
        return random.randint(1, 3)
    elif key == 'onOff':
        return random.choice([True, False])
    elif key == 'occupancy':
        return random.choice([True, False])
    elif key == 'power':
        return random.choice([True, False])
    elif key == 'powerConsumed':
        return random.uniform(80, 120)
    elif key == 'volume':
        return random.randint(0, 100)
    elif key == 'channel':
        return random.choice(["BBC", "CNN", "FOX", "ABC"])
    elif key == 'brightness':
        return random.randint(0, 100)
    elif key == 'lighting':
        return random.randint(0, 100)
    elif key == 'humidity':
        return random.randint(0, 100)
    elif key == 'speed':
        return random.randint(0, 5)
    elif key == 'color':
        return random.choice(["White", "Red", "Green", "Blue"])
    elif key == 'status':
        return random.choice(["Running", "Stopped", "Paused"])

# Function to append the updated values to 'Room-datacollect' Excel file
def append_to_datacollect(updated_values, file_path):
    # Add additional columns
    updated_values['Date'] = datetime.now().strftime('%Y-%m-%d')
    updated_values['Time'] = datetime.now().strftime('%H:%M:%S')

    # Check if file exists
    if os.path.exists(file_path):
        # Load existing data
        existing_data = pd.read_excel(file_path)
        # Append new data to existing data
        updated_values = pd.concat([existing_data, updated_values], ignore_index=True)
    
    # Write to Excel file
    updated_values.to_excel(file_path, index=False)


# Continuous update loop
i=0
while i<1:
    # Read the 'RoomScenario-smarthome' Excel file
    df = pd.read_excel('RoomScenario-smarthome.xlsx')

    # Update parameters with random values
    updated_df = update_parameters(df)

    # Define the file path for 'Room-datacollect' Excel file
    datacollect_file_path = os.path.join(os.getcwd(), 'Room-datacollect.xlsx')

    # Append the updated values to 'Room-datacollect' Excel file
    updated_df.to_excel('RoomScenario-smarthome.xlsx', index=False)
    append_to_datacollect(updated_df, 'Room-datacollect.xlsx')
    

    # Wait for 3 seconds
    time.sleep(3)
    i+=1


In [33]:
import pandas as pd
import json

def create_digital_twins_from_excel(excel_file_path, dt_client):
    # Read the Excel file
    df = pd.read_excel(excel_file_path)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        twin_id = row['ID (must be unique)']
        model_id = row['ModelID']
        initial_data = json.loads(row['Init Data'])  # Convert the string to a dictionary

        # Construct the digital twin object
        twin = {
            "$metadata": {
                "$model": model_id
            },
            **initial_data
        }

        # Make the call to create the digital twin
        dt_client.upsert_digital_twin(twin_id, twin)
        print(f"Created Digital Twin '{twin_id}' with ModelID '{model_id}'")

def create_relationships_from_excel(excel_file_path, dt_client):
    # Read the Excel file
    df = pd.read_excel(excel_file_path)

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        source_id = row['ID (must be unique)']
        target_id = row['Relationship (From)']
        relationship_name = row['Relationship Name']
        relationship_properties = {"contains": True}  # Adding relationship properties as specified in the Excel sheet

        # Construct the relationship object
        relationship = {
            "$targetId": target_id,
            "$relationshipName": relationship_name,
            **relationship_properties
        }

        # Make the call to create the relationship
        dt_client.upsert_relationship(source_id, relationship_name, relationship)
        print(f"Created Relationship '{relationship_name}' from '{source_id}' to '{target_id}'")

# Usage example:
# Assuming dt_client is an instance of DigitalTwinsClient
excel_file_path = "RoomScenario-smarthome.xlsx"
create_digital_twins_from_excel(excel_file_path, dt_client)
create_relationships_from_excel(excel_file_path, dt_client)


HttpResponseError: (ValidationFailed) Invalid twin specified
Code: ValidationFailed
Message: Invalid twin specified
Exception Details:	(ValidationFailed) Expected value of type boolean for occupancy on Twin.
	Code: ValidationFailed
	Message: Expected value of type boolean for occupancy on Twin.	(ValidationFailed) Expected value of type number for lighting on Twin.
	Code: ValidationFailed
	Message: Expected value of type number for lighting on Twin.	(ValidationFailed) Expected value of type number for humidity on Twin.
	Code: ValidationFailed
	Message: Expected value of type number for humidity on Twin.	(ValidationFailed) Expected value of type boolean for power on Twin.
	Code: ValidationFailed
	Message: Expected value of type boolean for power on Twin.

In [32]:
# Query Block

query_expression = 'SELECT * FROM digitaltwins'
query_result = dt_client.query_twins(query_expression)
print('DigitalTwins:')
for twin in query_result:
    print(twin['$dtId'])

DigitalTwins:
AC1
TV1
Fan1
Geyser1
Fridge1
Bulb1
Washer1
Thermostat1
Room1
